In [2]:
# 타자 경기별 기록 데이터

import pandas as pd

file = './data/KBO_2019_player_gamestats.csv'
raw = pd.read_csv(file, encoding = 'cp949')
raw.head()

,팀,이름,생일,일자,상대,결과,타순,P,선발,타수,...,희타,희비,타율,출루,장타,OPS,투구,avLI,RE24,WPA
0,두산,페르난데스,1988-04-27,03-23,한화,W 5:4,6,DH,1,4,...,0,0,0.500,0.500,0.750,1.250,19,1.98,1.65,0.429
1,두산,페르난데스,1988-04-27,03-24,한화,L 1:11,6,DH,1,2,...,0,0,0.333,0.333,0.500,0.833,4,0.77,-0.36,-0.038
2,두산,페르난데스,1988-04-27,03-26,키움,W 7:2,2,DH,1,2,...,0,0,0.250,0.400,0.375,0.775,16,1.56,0.98,0.146
3,두산,페르난데스,1988-04-27,03-27,키움,W 3:2,2,DH,1,4,...,0,0,0.417,0.500,0.500,1.000,11,1.53,1.29,0.189
4,두산,페르난데스,1988-04-27,03-28,키움,L 4:5,2,DH,1,3,...,0,0,0.333,0.474,0.400,0.874,23,2.04,-0.30,-0.166


In [3]:
# '상대' 컬럼에서 @가 들어있는건 원경경기를 뜻함

raw['상대'].unique()

array(['한화', '키움', '@삼성', 'KT', 'NC', '@롯데', '@LG', 'SK', '@KIA', '@키움',
       '롯데', '@한화', 'LG', 'KIA', '@NC', '삼성', '@SK', '@KT', '두산', '@두산'],
      dtype=object)

In [42]:
# 홈어웨이, 상대팀 컬럼으로 정리

opp_list = []
home_away_list = []

for opp in raw['상대']:
    if '@' in opp:
        home_away = '원정'
        opp = opp.replace('@','')
    else :
        home_away = '홈'
        
    opp_list.append(opp)
    home_away_list.append(home_away)
    
raw['홈/어웨이'] = home_away_list
raw['상대팀'] = opp_list

raw.head()
    

,팀,이름,생일,일자,상대,결과,타순,P,선발,타수,...,타율,출루,장타,OPS,투구,avLI,RE24,WPA,홈/어웨이,상대팀
0,두산,페르난데스,1988-04-27,03-23,한화,W 5:4,6,DH,1,4,...,0.500,0.500,0.750,1.250,19,1.98,1.65,0.429,홈,한화
1,두산,페르난데스,1988-04-27,03-24,한화,L 1:11,6,DH,1,2,...,0.333,0.333,0.500,0.833,4,0.77,-0.36,-0.038,홈,한화
2,두산,페르난데스,1988-04-27,03-26,키움,W 7:2,2,DH,1,2,...,0.250,0.400,0.375,0.775,16,1.56,0.98,0.146,홈,키움
3,두산,페르난데스,1988-04-27,03-27,키움,W 3:2,2,DH,1,4,...,0.417,0.500,0.500,1.000,11,1.53,1.29,0.189,홈,키움
4,두산,페르난데스,1988-04-27,03-28,키움,L 4:5,2,DH,1,3,...,0.333,0.474,0.400,0.874,23,2.04,-0.30,-0.166,홈,키움


In [43]:
# 상대팀별 성적 정리

data = raw.pivot_table(index =['팀', '이름', '생일','상대팀'],
               values = ['타수', '안타','홈런', '루타', '타점','볼넷', '사구','희비'],
               aggfunc = 'sum')

data.head()

루타  볼넷  사구  안타  타수  타점  홈런  희비
팀   이름  생일         상대팀                                
KIA 고영창 1989-02-24 LG    0   0   0   0   0   0   0   0
    김선빈 1989-12-18 KT    5   1   0   4  33   4   0   0
                   LG   15   5   0  12  47   6   0   0
                   NC   14   8   0  14  42   2   0   0
                   SK   10   6   0  10  36   4   0   1

In [44]:
cond = data['타수'] > 0
data = data[cond]

In [45]:
data = data.reset_index()
data.head()

,팀,이름,생일,상대팀,루타,볼넷,사구,안타,타수,타점,홈런,희비
0,KIA,김선빈,1989-12-18,KT,5,1,0,4,33,4,0,0
1,KIA,김선빈,1989-12-18,LG,15,5,0,12,47,6,0,0
2,KIA,김선빈,1989-12-18,NC,14,8,0,14,42,2,0,0
3,KIA,김선빈,1989-12-18,SK,10,6,0,10,36,4,0,1
4,KIA,김선빈,1989-12-18,두산,20,3,0,15,54,5,0,0


In [46]:
def cal_hit(df):
    '''
    - 타율 : 공을 쳐서 나가는 비율 --> 안타 / 타수
    - 출루율: 진루해서 나가는 비율 -->  (안타+볼넷+몸에맞는볼)/(타수+볼넷+몸에맞는볼+희생플라이)
    - 장타율 : 타율에 진루한 베이스 가중치 추가 -->   루타 / 타수
    '''
    
    df['타율'] = df['안타'] / df['타수']
    df['출루율'] = (df['안타'] + df['볼넷'] + df['사구']) / (df['타수'] + df['사구'] + df['희비'])
    df['장타율'] = df['루타'] / df['타수']
    df['OPS'] = df['출루율'] + df['장타율']
    return df

In [47]:
player_stats_opp = cal_hit(data)
player_stats_opp.head(10)

,팀,이름,생일,상대팀,루타,볼넷,사구,안타,타수,타점,홈런,희비,타율,출루율,장타율,OPS
0,KIA,김선빈,1989-12-18,KT,5,1,0,4,33,4,0,0,0.121212,0.151515,0.151515,0.303030
1,KIA,김선빈,1989-12-18,LG,15,5,0,12,47,6,0,0,0.255319,0.361702,0.319149,0.680851
2,KIA,김선빈,1989-12-18,NC,14,8,0,14,42,2,0,0,0.333333,0.523810,0.333333,0.857143
3,KIA,김선빈,1989-12-18,SK,10,6,0,10,36,4,0,1,0.277778,0.432432,0.277778,0.710210
4,KIA,김선빈,1989-12-18,두산,20,3,0,15,54,5,0,0,0.277778,0.333333,0.370370,0.703704
5,KIA,김선빈,1989-12-18,롯데,10,6,0,9,25,3,0,2,0.360000,0.555556,0.400000,0.955556
6,KIA,김선빈,1989-12-18,삼성,18,6,1,13,51,5,0,0,0.254902,0.384615,0.352941,0.737557
7,KIA,김선빈,1989-12-18,키움,22,1,0,16,49,4,1,0,0.326531,0.346939,0.448980,0.795918
8,KIA,김선빈,1989-12-18,한화,32,7,0,22,57,7,2,1,0.385965,0.500000,0.561404,1.061404
9,KIA,김주찬,1981-03-25,KT,5,1,1,5,22,1,0,0,0.227273,0.304348,0.227273,0.531621


In [48]:
# 특정 팀 상대로 OPS가 높은 타자

team = '두산'
cond = (player_stats_opp['상대팀'] == team) & (player_stats_opp['타수']>10)

player_stats_opp[cond].sort_values(by = 'OPS', ascending = False).head(5)

,팀,이름,생일,상대팀,루타,볼넷,사구,안타,타수,타점,홈런,희비,타율,출루율,장타율,OPS
22,KIA,나지완,1985-05-19,두산,11,7,0,5,17,4,1,1,0.294118,0.666667,0.647059,1.313725
538,NC,나성범,1989-10-03,두산,9,1,1,5,12,2,1,0,0.416667,0.538462,0.750000,1.288462
546,NC,노진혁,1989-07-15,두산,26,0,0,14,33,7,2,0,0.424242,0.424242,0.787879,1.212121
466,LG,페게로,1987-02-22,두산,15,1,0,7,19,8,2,0,0.368421,0.421053,0.789474,1.210526
1664,한화,호잉,1989-05-18,두산,30,11,0,18,54,5,2,0,0.333333,0.537037,0.555556,1.092593


In [50]:
# 롯데 상대

team = '롯데'

cond = (player_stats_opp['상대팀'] == team) & (player_stats_opp['타수']>20)
player_stats_opp[cond].sort_values(by = 'OPS', ascending = False).head()

,팀,이름,생일,상대팀,루타,볼넷,사구,안타,타수,타점,홈런,희비,타율,출루율,장타율,OPS
330,KT,황재균,1987-07-28,롯데,35,11,0,22,47,12,3,1,0.468085,0.687500,0.744681,1.432181
610,NC,양의지,1987-06-05,롯데,29,7,2,20,42,8,2,0,0.476190,0.659091,0.690476,1.349567
1433,키움,서건창,1989-08-22,롯데,26,3,0,21,39,7,0,0,0.538462,0.615385,0.666667,1.282051
574,NC,박석민,1985-06-22,롯데,39,12,1,18,54,19,5,1,0.333333,0.553571,0.722222,1.275794
137,KIA,최형우,1983-12-16,롯데,38,7,0,18,50,17,5,1,0.360000,0.490196,0.760000,1.250196


In [51]:
# 각 팀 대상 OPS 상위 5인

player_stats_opp['상대팀'].unique()


array(['KT', 'LG', 'NC', 'SK', '두산', '롯데', '삼성', '키움', '한화', 'KIA'],
      dtype=object)

In [53]:
hitter_df = pd.DataFrame()

for team in player_stats_opp['상대팀'].unique():
    cond = (player_stats_opp['상대팀'] == team) & (player_stats_opp['타수']>20)
    df = player_stats_opp[cond].sort_values(by = 'OPS', ascending = False).head()
    
    hitter_df = hitter_df.append(df)
    

In [54]:
hitter_df

,팀,이름,생일,상대팀,루타,볼넷,사구,안타,타수,타점,홈런,희비,타율,출루율,장타율,OPS
1401,키움,박병호,1986-07-10,KT,26,14,1,10,38,10,5,2,0.263158,0.609756,0.684211,1.293967
882,두산,박건우,1990-09-08,KT,32,9,0,16,46,11,2,1,0.347826,0.531915,0.695652,1.227567
463,LG,페게로,1987-02-22,KT,19,3,0,11,27,11,2,0,0.407407,0.518519,0.703704,1.222222
445,LG,조셉,1991-07-16,KT,18,1,0,9,23,6,3,0,0.391304,0.434783,0.782609,1.217391
1337,삼성,최영진,1988-05-10,KT,18,5,1,10,30,6,2,0,0.333333,0.516129,0.600000,1.116129
1375,키움,김하성,1995-10-17,LG,39,18,1,27,69,13,1,1,0.391304,0.647887,0.565217,1.213105
1609,한화,정근우,1982-10-02,LG,13,8,0,11,27,7,0,0,0.407407,0.703704,0.481481,1.185185
659,SK,고종욱,1989-01-11,LG,37,1,0,24,55,6,1,0,0.436364,0.454545,0.672727,1.127273
142,KIA,터커,1990-07-06,LG,19,7,0,15,36,9,1,1,0.416667,0.594595,0.527778,1.122372
792,SK,최정,1987-02-28,LG,42,4,4,21,62,15,5,1,0.338710,0.432836,0.677419,1.110255


In [55]:
hitter_df['이름'].unique()

array(['박병호', '박건우', '페게로', '조셉', '최영진', '김하성', '정근우', '고종욱', '터커', '최정',
       '오재일', '박승욱', '민병헌', '강백호', '김태균', '이원재', '강민호', '장영석', '양의지',
       '노진혁', '호잉', '이성열', '로맥', '박석민', '황재균', '서건창', '최형우', '김혜성', '이형종',
       '이정후', '안치홍', '스몰린스키', '윌슨', '샌즈', '러프', '최주환', '오지환'],
      dtype=object)

In [57]:
# 특정 팀상대 OPS 상위 타자들 대상으로 팀별 OPS

cond = []

for name in player_stats_opp['이름']:
    if name in hitter_df['이름'].unique():
        cond.append(True)
        
    else :
        cond.append(False)
        
player_stats_opp[cond]
        

,팀,이름,생일,상대팀,루타,볼넷,사구,안타,타수,타점,홈런,희비,타율,출루율,장타율,OPS
70,KIA,안치홍,1990-07-02,KT,15,1,0,14,49,6,0,0,0.285714,0.306122,0.306122,0.612245
71,KIA,안치홍,1990-07-02,LG,7,5,1,6,22,4,0,0,0.272727,0.521739,0.318182,0.839921
72,KIA,안치홍,1990-07-02,NC,20,3,0,12,39,7,2,0,0.307692,0.384615,0.512821,0.897436
73,KIA,안치홍,1990-07-02,SK,17,3,0,13,47,4,1,1,0.276596,0.333333,0.361702,0.695035
74,KIA,안치홍,1990-07-02,두산,15,2,0,12,40,2,0,0,0.300000,0.350000,0.375000,0.725000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1663,한화,호잉,1989-05-18,SK,20,2,1,15,55,7,1,0,0.272727,0.321429,0.363636,0.685065
1664,한화,호잉,1989-05-18,두산,30,11,0,18,54,5,2,0,0.333333,0.537037,0.555556,1.092593
1665,한화,호잉,1989-05-18,롯데,18,2,1,11,43,6,1,0,0.255814,0.318182,0.418605,0.736786
1666,한화,호잉,1989-05-18,삼성,20,3,1,12,51,8,2,1,0.235294,0.301887,0.392157,0.694044
